In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import os
import string
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
path = os.getcwd()
path

'e:\\Folder_apps\\NGODING\\Python\\PDM\\IPYNB-Code'

In [3]:
PATH = Path().cwd().parent
DDIR = PATH / 'datasets'
RDIR = PATH / 'result'

print(DDIR)

e:\Folder_apps\NGODING\Python\PDM\datasets


In [4]:
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

stop_word_1 = stopwords.words('indonesian')
stop_factory = StopWordRemoverFactory()
stop_word_2 = stop_factory.get_stop_words()

stop_word = list(set(stop_word_1 + stop_word_2))

In [5]:
# stop_word
stop_word

['apabila',
 'sama',
 'sekali',
 'ingat-ingat',
 'ke',
 'setidak-tidaknya',
 'tentang',
 'lamanya',
 'dini',
 'bapak',
 'berarti',
 'kerja',
 'asal',
 'bukanlah',
 'empat',
 'x',
 'waktu',
 'sebegitu',
 'cukuplah',
 'berapakah',
 'memerlukan',
 'mau',
 'menunjuki',
 'per',
 'tertentu',
 'mengerjakan',
 'sebegini',
 'memperkirakan',
 'hadap',
 'ditanyakan',
 'berapalah',
 'dong',
 'kemungkinan',
 'nyaris',
 'serupa',
 'dituturkannya',
 'menjadi',
 'jadilah',
 'pernah',
 'sedangkan',
 'kalau',
 'semuanya',
 'baik',
 'dikira',
 'sampai',
 'sendiri',
 'kinilah',
 'memang',
 'dapat',
 'turut',
 'hari',
 'ataupun',
 'lanjutnya',
 'memulai',
 'wahai',
 'dikerjakan',
 'berturut',
 'kapan',
 'hendaknya',
 'terjadi',
 'yakin',
 'telah',
 'lainnya',
 'kemungkinannya',
 'padahal',
 'kurang',
 'termasuk',
 'lima',
 'atas',
 'kepadanya',
 'diungkapkan',
 'seolah',
 'sesama',
 'menjawab',
 'anda',
 'bukannya',
 'dekat',
 'a',
 'demi',
 'dimaksudnya',
 'kok',
 'selalu',
 'jangankan',
 'terdahulu',
 'd

In [6]:
df = pd.read_excel(f'{DDIR}/Dataset B.xlsx')

In [7]:
df.sample(5)

,No,Reviews,Label
181,182,Pengiriman sangat lama,0
64,65,Gmna sih jam tngn ini🤔 ga bisa di atur lgi hmm...,0
167,168,Karnaa jamnya susah bgtt di pencet,0
178,179,tali nya agak kaku tapi nyalah semua,0
13,14,Muraah sekali kaca matanya tp syg ada cacat se...,0


In [8]:
df.drop(columns=['No'], inplace=True)

In [9]:
#Getting data 51 - 60
df_5 = df.iloc[50:60]

In [10]:
df_5

,Reviews,Label
50,"Agak sedih si warna jam nya keorange"" An gitu ...",0
51,"sebenernya jam nya cantik apalagi warnanya, cu...",0
52,Tidak recommended di toko ini padahal sudah la...,0
53,"Bagus bgt,cocok buat anak saya, pengiriman aga...",0
54,"Kecewa, karna banyak barang yg rusak yg dikiri...",0
55,"Pesen 2 yg idup cuma 1,barang rusak kecewa par...",0
56,awal percaya banget sama toko ini.eh kali saya...,0
57,Barang yg ditunggu2 ternyata tidak bagus ..bar...,0
58,Saya sudah kirim vidio nya dan ga dj kembalika...,0
59,Pengiriman lama mungkin krna 77 shope\nPengema...,1


In [11]:
# df_5.Reviews.iloc[0].lower().split(' ')
def converting(df):

    #\d+ untuk hilangkan digit
    #translate maketrans untuk hilangkan tanda baca & tautan
    #strip untuk hilangkan whitespaces
    data = df.str.lower().str.replace('\d+', '').str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).str.strip()
    return data

In [50]:
a = 'Bagus=bgs,krn=keren'
a = a.split(',')
print(a)

['Bagus=bgs', 'krn=keren']


In [52]:
for string in a:
    print(string.split('=')[0], string.split('=')[1])

Bagus bgs
krn keren


In [12]:
df_5['Clean'] = converting(df_5.Reviews)

C:\Users\Windows\AppData\Local\Temp/ipykernel_2764/3667073136.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data = df.str.lower().str.replace('\d+', '').str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).str.strip()
C:\Users\Windows\AppData\Local\Temp/ipykernel_2764/1838098756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Clean'] = converting(df_5.Reviews)


In [13]:
cols = ['Reviews','Clean','Label']
df_5 = df_5[cols]

In [14]:
TAG_RE = re.compile(r'<[^>]+>')
df_5.Clean = df_5.Clean.apply(lambda x:TAG_RE.sub('',re.sub('\s+',' ',x))) #removing kurung buka, kurung tutup, /n /t /b dsb

In [15]:
#stop_words
df_5.Clean = df_5.Clean.apply(lambda x: ' '.join([w for w in x.split() if w not in stop_word]))

In [16]:
#Stemming
df_5['Clean'] = df_5['Clean'].apply(lambda x:stemmer.stem(x))

In [17]:
def stemming(df):
    for x in range(len(df)):
        datax = df.iloc[x].split()
        for y in range(len(datax)):
            datax[y] = stemmer.stem(datax[y])
        df.iloc[x] = ' '.join(datax)
    return df

In [18]:
stemming(df_5.Clean)

c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


50    sedih si warna jam keorange an gitu sen yg bag...
51    sebenernya jam cantik warna dipake udah copot ...
52     recommended toko langgan kecewaaaaaa baaangeettt
53    bagus bgt cocok anak kirim tp gag pesan yg bs ...
54    kecewa karna barang yg rusak yg kirim auto jdi...
55    sen yg idup barang rusak kecewa parah kurir jg...
56    percaya banget toko eh anggap toko ga amanah p...
57    barang yg tunggu bagus buka kemas ehh udh copo...
58    kirim vidio ga dj kembali uang saran beli ga t...
59    kirim krna shope emas super jelek bgttt untung...
Name: Clean, dtype: object

In [19]:
df_5.to_excel(f'{RDIR}/Dataset B.xlsx')

In [20]:
import spacy
nlp = spacy.blank('id')

In [21]:
kata = 'my younger brother watching tv'
# for token in kata:
#     ori = token.text
#     lemma = token.lemma_  # token.lemma is integer index
#     print(ori, lemma)

In [22]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("care")

'care'

In [23]:
from kbbi import KBBI
cinta = KBBI('bepergian')
print(str(cinta).split())
import json
# print(json.dumps(cinta.serialisasi(),indent=2))
# print(cinta.__str__(contoh=False,terkait=False,fitur_pengguna=False))

['pergi', '»', 'be.per.gi.an', '(v)', 'berjalan', 'jauh:', 'tiap', 'hari', 'Minggu', 'banyak', 'orang', '~', 'ke', 'luar', 'kota']


In [24]:
list_unq = np.array([],dtype=str)

In [25]:
set(df_5.Reviews.values[0].split())

{'Agak',
 'Aja',
 'An',
 'agak',
 'aku',
 'bagus"',
 'gak',
 'gatau',
 'gitu',
 'jam',
 'keorange"',
 'nya',
 'orange',
 'padahal',
 'pertama',
 'pesenan',
 'putih',
 'sedih',
 'sekarang',
 'si',
 'warna',
 'yg'}

In [26]:
np.zeros((5,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [27]:
len(df_5.Clean.unique())

10

In [28]:
list_unique = np.array([],dtype=str)
for x in df_5.Clean.values:
    list_unique = np.append(list_unique, list(set(x.split())))
# print(df_5.Clean.values[0].split())
print(list_unique)

['keorange' 'putih' 'bagus' 'warna' 'yg' 'aja' 'orange' 'an' 'gatau'
 'sedih' 'sen' 'gak' 'si' 'gitu' 'jam' 'warna' 'yg' 'copot' 'pasang'
 'sebenernya' 'dipake' 'udah' 'lubang' 'cantik' 'jam' 'item' 'baaangeettt'
 'kecewaaaaaa' 'recommended' 'langgan' 'toko' 'teliti' 'pesan' 'yg' 'cek'
 'bgt' 'tp' 'kirim' 'bs' 'sih' 'gag' 'bagus' 'dgn' 'cocok' 'anak' 'barang'
 'buntung' 'order' 'rusak' 'yg' 'kirim' 'jual' 'untung' 'auto' 'karna'
 'kecewa' 'kapok' 'jdi' 'utk' 'ater' 'barang' 'kurir' 'rusak' 'yg' 'idup'
 'parah' 'tuju' 'ramah' 'sen' 'gk' 'kecewa' 'jga' 'eh' 'deh' 'dsini' 'ga'
 'bungkus' 'liat' 'jd' 'anggap' 'toko' 'jual' 'aj' 'pesan' 'percaya'
 'banget' 'amanah' 'belanja' 'coba' 'pcs' 'barang' 'situ' 'hp' 'rugi' 'rb'
 'zonk' 'barang' 'aja' 'yg' 'kemas' 'ehh' 'copot' 'setel' 'datang' 'buka'
 'udh' 'kecewa' 'bagus' 'na' 'jam' 'tunggu' 'kado' 'ga' 'vidio' 'uang'
 'saran' 'dj' 'kembali' 'kirim' 'beli' 'tanggung' 'warna' 'ku' 'jelek'
 'untung' 'smua' 'plastik' 'boss' 'ga' 'shope' 'rusak' 'gan

In [29]:
init_data = np.zeros((len(list_unique),len(df_5.Clean)),dtype=int)

In [30]:
onehot = pd.DataFrame(init_data)
onehot['Words'] = list_unique

In [31]:
df_5.index.values

array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59], dtype=int64)

In [32]:
for key,x in enumerate(df_5.index.values):
    onehot.rename(columns={key:x},inplace=True)

In [33]:
onehot.set_index('Words',inplace=True)

In [34]:
for key,value in df_5.Clean.iteritems():
    for key2,value2 in enumerate(list_unique):
        onehot[key].iloc[key2] = value.split().count(value2)

In [35]:
onehot

,50,51,52,53,54,55,56,57,58,59
Words,,,,,,,,,,
keorange,1,0,0,0,0,0,0,0,0,0
putih,1,0,0,0,0,0,0,0,0,2
bagus,1,0,0,1,0,0,0,1,0,0
warna,2,1,0,0,0,0,0,0,0,1
yg,1,1,0,1,2,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
putih,1,0,0,0,0,0,0,0,0,2
itu,0,0,0,0,0,0,0,0,0,1
emas,0,0,0,0,0,0,0,0,0,1


In [36]:
#IDF = number of columns / value in specific rows
import math
list_idf = np.array([],dtype=float)
for key,value in onehot.iterrows():
    try:
        value = round(math.log(value.sum(),len(onehot.columns)),3)
    except:
        value = 0
    list_idf = np.append(list_idf, value)

In [37]:
# onehot['IDF-Score'] = list_idf

In [38]:
#TF = value / columns sum
for x in onehot.columns:
    list_tf = np.array([],dtype=float)
    summ = onehot[x].sum()
    for y in range(len(onehot[x])):
        try:
            value = (onehot[x].iloc[y] / summ) * list_idf[y]
        except:
            value = 0
        list_tf = np.append(list_tf, value)
    onehot[f'{x} TF-IDF-Score'] = list_tf


In [39]:
math.log2(3/2)

0.5849625007211562

In [40]:
# for key,value in onehot.iterrows():
    # print(key)
onehot

,50,51,52,53,54,55,56,57,58,59,50 TF-IDF-Score,51 TF-IDF-Score,52 TF-IDF-Score,53 TF-IDF-Score,54 TF-IDF-Score,55 TF-IDF-Score,56 TF-IDF-Score,57 TF-IDF-Score,58 TF-IDF-Score,59 TF-IDF-Score
Words,,,,,,,,,,,,,,,,,,,,
keorange,1,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000
putih,1,0,0,0,0,0,0,0,0,2,0.012892,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.022186
bagus,1,0,0,1,0,0,0,1,0,0,0.012892,0.000000,0.0,0.018346,0.00000,0.000000,0.0,0.012553,0.0,0.000000
warna,2,1,0,0,0,0,0,0,0,1,0.032541,0.027364,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.014000
yg,1,1,0,1,2,1,0,1,0,0,0.022838,0.038409,0.0,0.032500,0.04122,0.028167,0.0,0.022237,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
putih,1,0,0,0,0,0,0,0,0,2,0.012892,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.022186
itu,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000
emas,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000


In [41]:
# list_tf = np.array([],dtype=float)
# for x in onehot.columns:
#     print(onehot[x].sum())
#     for y in onehot[x]:
#         list_tf = np.append(list_tf, y/onehot.shape[0])
    

    
# onehot['tf'] = list_tf
# onehot


In [42]:
for key,value in df_5.Clean.iteritems():
    # print(value.split())
    # print(key)
    for key2,value2 in enumerate(list_unique):
        # print(value2)
        onehot[key].iloc[key2] = value.split().count(value2)
            # print(x)
            # onehot.loc[key,key3] = value.split().count()

c:\Users\Windows\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [43]:
RDIR

WindowsPath('e:/Folder_apps/NGODING/Python/PDM/result')

In [44]:
onehot
onehot.to_excel(f'{RDIR}/Tugas-3-onehot.xlsx')

In [45]:
for (index,value),(value2) in zip(onehot.iterrows(),df_5.Clean.values):
    print(value.values[-1])
    
    # if(value in value2):
    #     onehot.iloc[key] = value2.count(key)

0.0
0.022186046511627904
0.0
0.013999999999999999
0.0
0.0
0.0
0.0
0.0
0.0


In [46]:
y = pd.get_dummies(df_5['Clean'].str.split(), prefix=df_5['Clean'].unique())

TypeError: unhashable type: 'list'

In [ ]:
y

,['sedih si warna jam keorange an gitu sen yg bagus aja putih gitu gak gatau warna orange gitu sedih si'\n 'sebenernya jam cantik warna dipake udah copot yg item pasang lubang'\n 'recommended toko langgan kecewaaaaaa baaangeettt'\n 'bagus bgt cocok anak kirim tp gag pesan yg bs sih gag cek dgn teliti'\n 'kecewa karna barang yg rusak yg kirim auto jdi kapok order barang utk jual untung buntung'\n 'sen yg idup barang rusak kecewa parah kurir jga ramah barang gk ater tuju'\n 'percaya banget toko eh anggap toko ga amanah pesan pcs pcs rugi rb jd rugi jual coba liat deh bungkus aj hp barang pcs bungkus situ ga belanja dsini'\n 'barang yg tunggu bagus buka kemas ehh udh copot aja setel jam na kecewa kado tunggu datang kecewa zonk'\n 'kirim vidio ga dj kembali uang saran beli ga tanggung'\n 'kirim krna shope emas super jelek bgttt untung brg rusak tolong baik boss jam ga plastik itu warna putih smua ku ganti biar putih ga kotor']_bagus bgt cocok anak kirim tp gag pesan yg bs sih gag cek dgn teliti,['sedih si warna jam keorange an gitu sen yg bagus aja putih gitu gak gatau warna orange gitu sedih si'\n 'sebenernya jam cantik warna dipake udah copot yg item pasang lubang'\n 'recommended toko langgan kecewaaaaaa baaangeettt'\n 'bagus bgt cocok anak kirim tp gag pesan yg bs sih gag cek dgn teliti'\n 'kecewa karna barang yg rusak yg kirim auto jdi kapok order barang utk jual untung buntung'\n 'sen yg idup barang rusak kecewa parah kurir jga ramah barang gk ater tuju'\n 'percaya banget toko eh anggap toko ga amanah pesan pcs pcs rugi rb jd rugi jual coba liat deh bungkus aj hp barang pcs bungkus situ ga belanja dsini'\n 'barang yg tunggu bagus buka kemas ehh udh copot aja setel jam na kecewa kado tunggu datang kecewa zonk'\n 'kirim vidio ga dj kembali uang saran beli ga tanggung'\n 'kirim krna shope emas super jelek bgttt untung brg rusak tolong baik boss jam ga plastik itu warna putih smua ku ganti biar putih ga kotor']_barang yg tunggu bagus buka kemas ehh udh copot aja setel jam na kecewa kado tunggu datang kecewa zonk,['sedih si warna jam keorange an gitu sen yg bagus aja putih gitu gak gatau warna orange gitu sedih si'\n 'sebenernya jam cantik warna dipake udah copot yg item pasang lubang'\n 'recommended toko langgan kecewaaaaaa baaangeettt'\n 'bagus bgt cocok anak kirim tp gag pesan yg bs sih gag cek dgn teliti'\n 'kecewa karna barang yg rusak yg kirim auto jdi kapok order barang utk jual untung buntung'\n 'sen yg idup barang rusak kecewa parah kurir jga ramah barang gk ater tuju'\n 'percaya banget toko eh anggap toko ga amanah pesan pcs pcs rugi rb jd rugi jual coba liat deh bungkus aj hp barang pcs bungkus situ ga belanja dsini'\n 'barang yg tunggu bagus buka kemas ehh udh copot aja setel jam na kecewa kado tunggu datang kecewa zonk'\n 'kirim vidio ga dj kembali uang saran beli ga tanggung'\n 'kirim krna shope emas super jelek bgttt untung brg rusak tolong baik boss jam ga plastik itu warna putih smua ku ganti biar putih ga kotor']_kecewa karna barang yg rusak yg kirim auto jdi kapok order barang utk jual untung buntung,['sedih si warna jam keorange an gitu sen yg bagus aja putih gitu gak gatau warna orange gitu sedih si'\n 'sebenernya jam cantik warna dipake udah copot yg item pasang lubang'\n 'recommended toko langgan kecewaaaaaa baaangeettt'\n 'bagus bgt cocok anak kirim tp gag pesan yg bs sih gag cek dgn teliti'\n 'kecewa karna barang yg rusak yg kirim auto jdi kapok order barang utk jual untung buntung'\n 'sen yg idup barang rusak kecewa parah kurir jga ramah barang gk ater tuju'\n 'percaya banget toko eh anggap toko ga amanah pesan pcs pcs rugi rb jd rugi jual coba liat deh bungkus aj hp barang pcs bungkus situ ga belanja dsini'\n 'barang yg tunggu bagus buka kemas ehh udh copot aja setel jam na kecewa kado tunggu datang kecewa zonk'\n 'kirim vidio ga dj kembali uang saran beli ga tanggung'\n 'kirim krna shope emas super jelek bgttt untung brg rusak tolong baik boss jam ga plastik itu warna putih smua ku ganti biar putih ga k